In [1]:
import pandas as pd
import numpy as np
import datatable as dt

### Import Data

In [2]:
Accident = dt.fread("dft-accident-data/Accidents0515.csv").to_pandas()
Casualty = dt.fread("dft-accident-data/Clean Data/Casualties.csv").to_pandas()
Vehicle = dt.fread("dft-accident-data/Clean Data/Vehicles.csv").to_pandas()

### Merge the Three Raw Datas

In [3]:
Total = Accident.merge(Casualty, how = "outer").merge(Vehicle, how = "left")

In [4]:
Total.shape

(2402909, 66)

In [5]:
# Check Missing
Total.isnull().sum()[Total.isnull().sum() != 0]

Location_Easting_OSGR     183
Location_Northing_OSGR    183
Longitude                 183
Latitude                  183
dtype: int64

### Replace Values with Relational Datasets

In [6]:
def label_map(column_name,file) :
    df = pd.read_csv("dft-accident-data/contextCSVs/" + file)
    DICT = dict(zip(df[df.columns[0]], df[df.columns[1]]))
    global Total
    Total[column_name] = Total[column_name].map(DICT)

In [7]:
label_map("Accident_Severity", "Accident_Severity.csv")
label_map("Age_Band_of_Driver","Age_Band.csv")   #Age_Band
label_map("Age_Band_of_Casualty","Age_Band.csv") #Age_Band
label_map("Casualty_Class","Casualty_Class.csv")
label_map("Casualty_Severity","Casualty_Severity.csv")
label_map("Casualty_Type","Casualty_Type.csv")
label_map("Day_of_Week", "Day_of_Week.csv")
label_map("Journey_Purpose_of_Driver", "Journey_Purpose.csv")
label_map("Junction_Control", "Junction_Control.csv")
label_map("Junction_Detail", "Junction_Detail.csv")
label_map("Junction_Location", "Junction_Location.csv")
label_map("Light_Conditions", "Light_Conditions.csv")
label_map("Local_Authority_(District)", "Local_Authority_District.csv")
label_map("Local_Authority_(Highway)", "Local_Authority_Highway.csv")
label_map("Pedestrian_Crossing-Human_Control", "Ped_Cross_Human.csv")
label_map("Pedestrian_Crossing-Physical_Facilities", "Ped_Cross_Physical.csv")
label_map("Pedestrian_Location", "Ped_Location.csv")
label_map("Pedestrian_Movement", "Ped_Movement.csv")
label_map("1st_Point_of_Impact", "Point_of_Impact.csv")
label_map("Police_Force","Police_Force.csv")
label_map("Did_Police_Officer_Attend_Scene_of_Accident", "Police_Officer_Attend.csv")
label_map("1st_Road_Class", "Road_Class.csv") #Road_Class
label_map("2nd_Road_Class", "Road_Class.csv") #Road_Class
label_map("Road_Type", "Road_Type.csv")
label_map("Sex_of_Driver", "Sex_of_Driver.csv")
label_map("Sex_of_Casualty", "Sex_of_Driver.csv")
label_map("Urban_or_Rural_Area", "Urban_Rural.csv")
label_map("Vehicle_Location-Restricted_Lane", "Vehicle_Location.csv")
label_map("Vehicle_Manoeuvre", "Vehicle_Manoeuvre.csv")
label_map("Vehicle_Type", "Vehicle_Type.csv")

### Create Response Term

In [8]:
def Cal_AS_CS(SERIES) :
    DICT = {"Fatal" : 3, "Serious" : 2 , "Slight" : 1}
    Numbers = SERIES.map(DICT)
    return Numbers

AS = Cal_AS_CS(Total.Accident_Severity)
CS = Cal_AS_CS(Total.Casualty_Severity)

Total["Diff_ASCS"] = AS - CS

Total["LuckyOrNot"] = Total.Diff_ASCS.map({0:"Lucky", 1: "Unlucky", 2 : "Unlucky"})
Total.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,Diff_ASCS,LuckyOrNot
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,Metropolitan Police,Serious,1,1,04/01/2005,...,Female,74,66 - 75,-1,-1,-1,7,1,0,Lucky
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,Metropolitan Police,Slight,1,1,05/01/2005,...,Male,42,36 - 45,8268,2,3,-1,-1,0,Lucky
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,Metropolitan Police,Slight,2,1,06/01/2005,...,Male,62,56 - 65,1762,1,6,1,1,0,Lucky
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,Metropolitan Police,Slight,1,1,07/01/2005,...,Female,49,46 - 55,1769,1,4,2,1,0,Lucky
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,Metropolitan Police,Slight,1,1,10/01/2005,...,Male,49,46 - 55,85,1,10,-1,-1,0,Lucky


### Confirm Column Types

In [9]:
Total.drop(columns = ["Location_Easting_OSGR","Location_Northing_OSGR", "Longitude", "Latitude"], inplace = True)
Total["Journey_Purpose_of_Driver"] = Total.Journey_Purpose_of_Driver.apply(lambda x : "Unknown" if (x == "Other/Not known (2005-10)") or (x == "Not known") else x)
Total["Junction_Control"] = Total.Junction_Control.apply(lambda x : "Unknown" if x == "Total missing or out of range" else x)
Total["2nd_Road_Class"] = Total["2nd_Road_Class"].apply(lambda x : "Unknown" if x == "" else x)

In [10]:
Total.replace(to_replace = [-1, "Unknown"], value = np.nan, inplace = True)

In [11]:
def to_string(var) :
    global Total 
    Total[var] = Total[var].astype(str)

    
to_string("Weather_Conditions")
to_string("Road_Surface_Conditions")
to_string("Special_Conditions_at_Site")
to_string("Carriageway_Hazards")
to_string("Pedestrian_Road_Maintenance_Worker")
to_string("Casualty_Home_Area_Type")
to_string("Skidding_and_Overturning")
to_string("Hit_Object_in_Carriageway")
to_string("Vehicle_Leaving_Carriageway")
to_string("Hit_Object_off_Carriageway")
to_string("Was_Vehicle_Left_Hand_Drive?")
to_string("Propulsion_Code")
to_string("Driver_IMD_Decile")
to_string("Driver_Home_Area_Type")
to_string("Diff_ASCS")

In [12]:
Total.head()

,Accident_Index,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),...,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,Diff_ASCS,LuckyOrNot
0,200501BS00001,Metropolitan Police,Serious,1,1,04/01/2005,Tuesday,17:42,Kensington and Chelsea,Kensington and Chelsea,...,Female,74.0,66 - 75,NaN,nan,NaN,7.0,1.0,0,Lucky
1,200501BS00002,Metropolitan Police,Slight,1,1,05/01/2005,Wednesday,17:36,Kensington and Chelsea,Kensington and Chelsea,...,Male,42.0,36 - 45,8268.0,2.0,3.0,nan,nan,0,Lucky
2,200501BS00003,Metropolitan Police,Slight,2,1,06/01/2005,Thursday,00:15,Kensington and Chelsea,Kensington and Chelsea,...,Male,62.0,56 - 65,1762.0,1.0,6.0,1.0,1.0,0,Lucky
3,200501BS00004,Metropolitan Police,Slight,1,1,07/01/2005,Friday,10:35,Kensington and Chelsea,Kensington and Chelsea,...,Female,49.0,46 - 55,1769.0,1.0,4.0,2.0,1.0,0,Lucky
4,200501BS00005,Metropolitan Police,Slight,1,1,10/01/2005,Monday,21:13,Kensington and Chelsea,Kensington and Chelsea,...,Male,49.0,46 - 55,85.0,1.0,10.0,nan,nan,0,Lucky


In [13]:
pd.options.display.float_format = "{:.2f}".format
Total.select_dtypes(include = ["float64", "int"]).describe()

,1st_Road_Number,2nd_Road_Number,Age_of_Casualty,Car_Passenger,Bus_or_Coach_Passenger,Towing_and_Articulation,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle
count,2402907.00,2380211.00,2353995.00,2402129.00,2402846.00,2402663.00,2308445.00,1807804.00,1735458.00
mean,1015.73,382.93,35.23,0.28,0.09,0.02,37.75,1767.12,7.44
std,1826.69,1305.67,18.37,0.59,0.56,0.23,16.06,1486.62,4.61
min,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
25%,0.00,0.00,21.00,0.00,0.00,0.00,24.00,1242.00,4.00
50%,134.00,0.00,32.00,0.00,0.00,0.00,35.00,1590.00,7.00
75%,724.00,0.00,47.00,0.00,0.00,0.00,48.00,1956.00,10.00
max,9999.00,9999.00,104.00,2.00,4.00,5.00,100.00,99999.00,111.00


### Output Data

In [14]:
Total.to_csv("Response Data.csv", index = False)